# Herkenning van benoemde entiteiten (NER)

Dit notebook is afkomstig uit [AI for Beginners Curriculum](http://aka.ms/ai-beginners).

In dit voorbeeld leren we hoe we een NER-model kunnen trainen op de [Annotated Corpus for Named Entity Recognition](https://www.kaggle.com/datasets/abhinavwalia95/entity-annotated-corpus) dataset van Kaggle. Voordat we verder gaan, download alsjeblieft het bestand [ner_dataset.csv](https://www.kaggle.com/datasets/abhinavwalia95/entity-annotated-corpus?resource=download&select=ner_dataset.csv) naar de huidige map.


In [62]:
import pandas as pd
from tensorflow import keras
import numpy as np

## Het voorbereiden van de dataset

We beginnen met het inlezen van de dataset in een dataframe. Als je meer wilt leren over het gebruik van Pandas, bekijk dan een [les over gegevensverwerking](https://github.com/microsoft/Data-Science-For-Beginners/tree/main/2-Working-With-Data/07-python) in onze [Data Science voor Beginners](http://aka.ms/datascience-beginners).


In [3]:
df = pd.read_csv('ner_dataset.csv',encoding='unicode-escape')
df.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


Laten we unieke tags verkrijgen en opzoekwoordenboeken maken die we kunnen gebruiken om tags om te zetten in klasnummers:


In [4]:
tags = df.Tag.unique()
tags

array(['O', 'B-geo', 'B-gpe', 'B-per', 'I-geo', 'B-org', 'I-org', 'B-tim',
       'B-art', 'I-art', 'I-per', 'I-gpe', 'I-tim', 'B-nat', 'B-eve',
       'I-eve', 'I-nat'], dtype=object)

In [8]:
id2tag = dict(enumerate(tags))
tag2id = { v : k for k,v in id2tag.items() }

id2tag[0]

'O'

Nu moeten we hetzelfde doen met vocabulaire. Voor de eenvoud zullen we vocabulaire creëren zonder rekening te houden met woordfrequentie; in het echte leven zou je misschien de Keras-vectorizer willen gebruiken en het aantal woorden beperken.


In [14]:
vocab = set(df['Word'].apply(lambda x: x.lower()))
id2word = { i+1 : v for i,v in enumerate(vocab) }
id2word[0] = '<UNK>'
vocab.add('<UNK>')
word2id = { v : k for k,v in id2word.items() }

We moeten een dataset van zinnen maken voor training. Laten we door de originele dataset lopen en alle individuele zinnen scheiden in `X` (lijsten van woorden) en `Y` (lijst van tokens):


In [41]:
X,Y = [],[]
s,t = [],[]
for i,row in df[['Sentence #','Word','Tag']].iterrows():
    if pd.isna(row['Sentence #']):
        s.append(row['Word'])
        t.append(row['Tag'])
    else:
        if len(s)>0:
            X.append(s)
            Y.append(t)
        s,t = [row['Word']],[row['Tag']]
X.append(s)
Y.append(t)


In [93]:
def vectorize(seq):
    return [word2id[x.lower()] for x in seq]

def tagify(seq):
    return [tag2id[x] for x in seq]

Xv = list(map(vectorize,X))
Yv = list(map(tagify,Y))

Xv[0], Yv[0]

([10386,
  23515,
  4134,
  29620,
  7954,
  13583,
  21193,
  12222,
  27322,
  18258,
  5815,
  15880,
  5355,
  25242,
  31327,
  18258,
  27067,
  23515,
  26444,
  14412,
  358,
  26551,
  5011,
  30558],
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0])

Voor eenvoud zullen we alle zinnen opvullen met 0 tokens tot de maximale lengte. In het echte leven zouden we een slimmere strategie kunnen gebruiken en sequenties alleen binnen één minibatch opvullen.


In [51]:
X_data = keras.preprocessing.sequence.pad_sequences(Xv,padding='post')
Y_data = keras.preprocessing.sequence.pad_sequences(Yv,padding='post')

## Het definiëren van een Token Classificatie Netwerk

We zullen een tweelaags bidirectioneel LSTM-netwerk gebruiken voor tokenclassificatie. Om een dense classifier toe te passen op elk van de outputs van de laatste LSTM-laag, maken we gebruik van de `TimeDistributed` constructie. Deze constructie herhaalt dezelfde dense laag voor elk van de outputs van de LSTM bij elke stap:


In [94]:
maxlen = X_data.shape[1]
vocab_size = len(vocab)
num_tags = len(tags)
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size, 300, input_length=maxlen),
    keras.layers.Bidirectional(keras.layers.LSTM(units=100, activation='tanh', return_sequences=True)),
    keras.layers.Bidirectional(keras.layers.LSTM(units=100, activation='tanh', return_sequences=True)),
    keras.layers.TimeDistributed(keras.layers.Dense(num_tags, activation='softmax'))
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 104, 300)          9545400   
                                                                 
 bidirectional_6 (Bidirectio  (None, 104, 200)         320800    
 nal)                                                            
                                                                 
 bidirectional_7 (Bidirectio  (None, 104, 200)         240800    
 nal)                                                            
                                                                 
 time_distributed_3 (TimeDis  (None, 104, 17)          3417      
 tributed)                                                       
                                                                 
Total params: 10,110,417
Trainable params: 10,110,417
Non-trainable params: 0
__________________________________________

Houd er rekening mee dat we hier expliciet `maxlen` specificeren voor ons dataset - als we willen dat het netwerk variabele lengtes van sequenties aankan, moeten we iets slimmer zijn bij het definiëren van het netwerk.

Laten we nu het model trainen. Voor de snelheid trainen we slechts één epoch, maar je kunt proberen langer te trainen. Ook kun je overwegen een deel van het dataset apart te houden als trainingsdataset, om de validatienauwkeurigheid te observeren.


In [57]:
model.fit(X_data,Y_data)

1499/1499 [==============================] - 740s 488ms/step - loss: 0.0667 - acc: 0.9841


## Het resultaat testen

Laten we nu bekijken hoe ons model voor entiteitsherkenning werkt op een voorbeeldzin:


In [91]:
sent = 'John Smith went to Paris to attend a conference in cancer development institute'
words = sent.lower().split()
v = keras.preprocessing.sequence.pad_sequences([[word2id[x] for x in words]],padding='post',maxlen=maxlen)
res = model(v)[0]

In [92]:
r = np.argmax(res.numpy(),axis=1)
for i,w in zip(r,words):
    print(f"{w} -> {id2tag[i]}")

john -> B-per
smith -> I-per
went -> O
to -> O
paris -> B-geo
to -> O
attend -> O
a -> O
conference -> O
in -> O
cancer -> B-org
development -> I-org
institute -> I-org


## Belangrijkste punten

Zelfs een eenvoudig LSTM-model laat redelijke resultaten zien bij NER. Echter, om veel betere resultaten te behalen, kun je beter grote, vooraf getrainde taalmodellen zoals BERT gebruiken. Het trainen van BERT voor NER met behulp van de Huggingface Transformers-bibliotheek wordt [hier](https://huggingface.co/course/chapter7/2?fw=pt) beschreven.



---

**Disclaimer**:  
Dit document is vertaald met behulp van de AI-vertalingsservice [Co-op Translator](https://github.com/Azure/co-op-translator). Hoewel we streven naar nauwkeurigheid, dient u zich ervan bewust te zijn dat geautomatiseerde vertalingen fouten of onnauwkeurigheden kunnen bevatten. Het originele document in zijn oorspronkelijke taal moet worden beschouwd als de gezaghebbende bron. Voor cruciale informatie wordt professionele menselijke vertaling aanbevolen. Wij zijn niet aansprakelijk voor eventuele misverstanden of verkeerde interpretaties die voortvloeien uit het gebruik van deze vertaling.
